<a href="https://colab.research.google.com/github/Engineer-D/My-Practices/blob/main/Semantic_Search_V0_SBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=01859f3ca8c2f6b93caeb9b3b493e59f37345ba9c221fb6111b101d273330cdc
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 53.4 MB/s eta 0:00:00


## Comment Generation

In [ ]:
from transformers import pipeline
generator = pipeline('text-generation', model = 'gpt2')
generator("Hello, I'm a language model", max_length = 30, num_return_sequences=3)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language model\n\nThis one feels like an afterthought, but as you can tell that's not all. What's more"},
 {'generated_text': "Hello, I'm a language modeler and a web developer myself. I've come up with many great design ideas and have been given a lot of"},
 {'generated_text': "Hello, I'm a language modeler. This tutorial will demonstrate how you can use it when you are debugging.\n\n#!/usr/bin"}]

In [ ]:
ss = generator("Hello, I'm a language model", max_length = 30, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
ss

[{'generated_text': "Hello, I'm a language model. And, I've got a really cool concept of how I work. When you think of language models of languages"},
 {'generated_text': 'Hello, I\'m a language model expert." "Uh…" His eyes were wide open and his heart beat with passion. "If you\'ve heard of'},
 {'generated_text': "Hello, I'm a language modeler. A lot of the best programming languages are compiled and statically linked to native code, and I prefer to keep"}]

In [ ]:
print(ss[0]['generated_text'])

Hello, I'm a language model. And, I've got a really cool concept of how I work. When you think of language models of languages


In [ ]:
def generate_comment(Question, Answer):
  '''
  Function that returns a generated comment based on question and response
  '''

  #Prompt
  prompt = \
  f"""
  Given
  Question: {Question}
  Answer: {Answer}
  Generate a comment to back up your answer
  """
  #comment Generation
  generator = pipeline('text-generation', model = 'gpt2')
  comment = generator(prompt, max_length = 30, num_return_sequences=3)

  return comment

In [ ]:
question = "Is there a clearly available mechanism (e.g. webpage, etc.) for data subjects that explains how to contact your organisation to pursue issues relating to personal data?( Article 2.5)"
answer = "Theres a page in our website that discuss about that"

comments = generate_comment(question, answer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 75, but `max_length` is set to 30. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


In [ ]:
print(comments[0]['generated_text'])


  Given
  Question: Is there a clearly available mechanism (e.g. webpage, etc.) for data subjects that explains how to contact your organisation to pursue issues relating to personal data?( Article 2.5)
  Answer: Theres a page in our website that discuss about that
  Generate a comment to back up your answer
   


## Semantic Search

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("/content/QuestionTest.csv", delimiter="\t", header = None)

In [ ]:
df.columns = ["Question", "Answer"]
df['embeddings'] = 0
df['meta'] = 0

In [ ]:
df.head()

,Question,Answer,embeddings,meta
0,Do you have a member of your organization with...,Yes,0,0
1,Is a background check required for all employe...,No,0,0
2,Does the organization have written information...,No,0,0
3,"If yes, please provide copies when respondi...",See Comments,0,0
4,Does the organization have a written password ...,Yes,0,0


In [ ]:
from tqdm.auto import tqdm

In [ ]:
from sentence_transformers import SentenceTransformer,util

# initialize sentence transformer model
# all-MiniLM-L6-v2
model = SentenceTransformer('all-MiniLM-L6-v2')
model.save("/content/drive/MyDrive/SbertV0")

# create sentence embeddings
sentence_embeddings = model.encode(df['Question'])
sentence_embeddings.shape

(91, 384)

In [ ]:
import faiss
d = sentence_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.is_trained

True

In [ ]:
index.add(sentence_embeddings)
index.ntotal

91

In [ ]:
query = list(df['Question'][[2,34,52,12,77,3]])

In [ ]:
k = 3
xq = model.encode(query)

In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 2  6 14]
 [34 76 43]
 [52 45 82]
 [12 10 25]
 [77  1 61]
 [ 3 15 21]]
CPU times: user 3.4 ms, sys: 0 ns, total: 3.4 ms
Wall time: 3.31 ms


In [ ]:
for i in tqdm(range(len(I))):
  print(f"Question: {query[i]}")
  
  for similar_question in I[i]:
    print(f"Similar Question: {df['Question'].iloc[similar_question]}")
    answer = df[df["Question"] == df['Question'].iloc[similar_question]]["Answer"][similar_question]
    print(f'Answer: {answer}')
  print()

  0%|          | 0/6 [00:00<?, ?it/s]

Question: Does the organization have written information security policies?
Similar Question: Does the organization have written information security policies?
Answer: No
Similar Question: Do all staff have access to relevant Information Security Policies and are they willing to comply with the policies as well as the data protection guidelines?
Answer: Yes
Similar Question: Does the organization have security measures in place for data protection?
Answer: No

Question: What security monitoring procedures are in place?
Similar Question: What security monitoring procedures are in place?
Answer: See Comments
Similar Question: What are the physical Security Check and Access Controls put in place by your organisation?
Answer: See Comments
Similar Question: Are servers configured to capture who accessed a system and what changes were made?
Answer: No

Question: Does the organization have a formal Incident Response plan?
Similar Question: Does the organization have a formal Incident Response

With Confidence

In [ ]:
confidences = []
similar_questions = []
answers = []

In [ ]:
for i in tqdm(range(len(I))):
  print(f"Question: {query[i]}")
  
  vecs = np.zeros((k, d))
  for j, val in enumerate(I[i].tolist()):
    vecs[j, :] = index.reconstruct(val)

  cos_sim = util.cos_sim(xq[i].astype(np.float32), vecs.astype(np.float32))

  confidence = cos_sim[0].numpy()*100
  confidences.append(confidence)

  similar_question = []
  answer_list = []
  for i,similar_question_index in enumerate(I[i]):
    similar_question.append(df['Question'].iloc[similar_question_index])
    print(f"{confidence[i]:.1f}: {df['Question'].iloc[similar_question_index]}")
    answer = df[df["Question"] == df['Question'].iloc[similar_question_index]]["Answer"][similar_question_index]
    answer_list.append(answer)
    print(f'Answer: {answer}')
  print()
  similar_questions.append(similar_question)
  answers.append(answer_list)

  0%|          | 0/6 [00:00<?, ?it/s]

Question: Does the organization have written information security policies?
100.0: Does the organization have written information security policies?
Answer: No
77.8: Do all staff have access to relevant Information Security Policies and are they willing to comply with the policies as well as the data protection guidelines?
Answer: Yes
75.3: Does the organization have security measures in place for data protection?
Answer: No

Question: What security monitoring procedures are in place?
100.0: What security monitoring procedures are in place?
Answer: See Comments
70.2: What are the physical Security Check and Access Controls put in place by your organisation?
Answer: See Comments
60.0: Are servers configured to capture who accessed a system and what changes were made?
Answer: No

Question: Does the organization have a formal Incident Response plan?
100.0: Does the organization have a formal Incident Response plan?
Answer: Yes
54.1: Does the organization have disaster recovery plans for d

In [ ]:
confidences

[array([100.     ,  77.77159,  75.28353], dtype=float32),
 array([100.000015,  70.20449 ,  60.021294], dtype=float32),
 array([99.99999 , 54.073597, 51.825172], dtype=float32),
 array([100.      ,  70.77478 ,  43.139503], dtype=float32),
 array([100.      ,  64.57018 ,  62.891422], dtype=float32),
 array([100.000015,  52.302967,  52.302967], dtype=float32)]

In [ ]:
similar_questions

[['Does the organization have written information security policies?',
  'Do all staff have access to relevant Information Security Policies and are they willing to comply with the policies as well as the data protection guidelines?',
  'Does the organization have security measures in place for data protection?'],
 ['What security monitoring procedures are in place?',
  'What are the physical Security Check and Access Controls put in place by your organisation?',
  'Are servers configured to capture who accessed a system and what changes were made?'],
 ['Does the organization have a formal Incident Response plan?',
  'Does the organization have disaster recovery plans for data processing facilities?',
  'Are there procedures in place that specify when authorities (e.g. law enforcement) should be contacted, and how identified information security incidents should be reported in a timely manner if it is suspected that laws may have been broken?'],
 ['   Are system and security patches te

In [ ]:
answers

[['No', 'Yes', 'No'],
 ['See Comments', 'See Comments', 'No'],
 ['Yes', 'Yes', 'Yes'],
 ['Yes', 'Yes', 'See Comments'],
 ['No', 'No', 'No'],
 ['See Comments', 'See Comments', 'See Comments']]

In [ ]:
r_query = ['Is there any information security policy at your company?',
 'How do you handle security monitoring?',
 'As regards formal Incident response plans do you have any?',
 'Before deploying to production environment do test for security and security patches?',
 'Is there any background check on new hires or potentials',
 'provide copies when responding to this accreditation']

In [ ]:
k = 3
xq = model.encode(r_query)

In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 2  6 14]
 [34 76 82]
 [52 82 45]
 [12 10 76]
 [77 61  1]
 [ 3 58 36]]
CPU times: user 544 µs, sys: 0 ns, total: 544 µs
Wall time: 555 µs


In [ ]:
for i in tqdm(range(len(I))):
  print(f"Question: {query[i]}")
  for similar_question in I[i]:
    print(f"Similar Question: {df['Question'].iloc[similar_question]}")
    answer = df[df["Question"] == df['Question'].iloc[similar_question]]["Answer"][similar_question]
    print(f'Answer: {answer}')
  print()

  0%|          | 0/6 [00:00<?, ?it/s]

Question: Does the organization have written information security policies?
Similar Question: Does the organization have written information security policies?
Answer: No
Similar Question: Do all staff have access to relevant Information Security Policies and are they willing to comply with the policies as well as the data protection guidelines?
Answer: Yes
Similar Question: Does the organization have security measures in place for data protection?
Answer: No

Question: What security monitoring procedures are in place?
Similar Question: What security monitoring procedures are in place?
Answer: See Comments
Similar Question: What are the physical Security Check and Access Controls put in place by your organisation?
Answer: See Comments
Similar Question: Are there procedures in place that specify when authorities (e.g. law enforcement) should be contacted, and how identified information security incidents should be reported in a timely manner if it is suspected that laws may have been br

Ideas:  

(1) Change the model so it does better on unseen data

Evalution Side

In [ ]:
import numpy as np

In [ ]:
vecs = np.zeros((k, d))
for i, val in enumerate(I[0].tolist()):
  vecs[i, :] = index.reconstruct(val)

In [ ]:
vecs.shape

(3, 384)

In [ ]:
from sentence_transformers import util

In [ ]:
vecs.astype(np.float)

<ipython-input-56-75bfe63b2b81>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  vecs.astype(np.float)


array([[ 0.00118882,  0.0448921 , -0.02852769, ..., -0.07402352,
         0.04142419, -0.02981823],
       [-0.00204194, -0.00702049,  0.01854258, ..., -0.07606407,
         0.01745052,  0.02866534],
       [-0.03782068,  0.0658571 , -0.01043941, ..., -0.1193842 ,
         0.06754075, -0.02628859]])

In [ ]:
util.cos_sim(xq[0].astype(np.float), vecs.astype(np.float))

<ipython-input-57-78e221a405fc>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  util.cos_sim(xq[0].astype(np.float), vecs.astype(np.float))


tensor([[0.7803, 0.7351, 0.7054]], dtype=torch.float64)

In [ ]:
cos_sim = util.cos_sim(xq[0].astype(np.float), vecs.astype(np.float))

<ipython-input-63-cbee84a6f364>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cos_sim = util.cos_sim(xq[0].astype(np.float), vecs.astype(np.float))


In [ ]:
ss = cos_sim[0].numpy()*100

ss

array([100.        ,  77.77158859,  75.28352936])

In [ ]:
print(f"{ss[2]:.2f}")

75.28


In [ ]:
arr = np.empty([5, 3])
arr = np.append(arr[0], np.array([[13,15,17]])) 
# app_arr = np.append(app_arr, [[13,15,17]], axis = 1) 
arr

array([8.17313905e-316, 3.90311860e-322, 0.00000000e+000, 1.30000000e+001,
       1.50000000e+001, 1.70000000e+001])

Testing Deployement Code

In [ ]:
#import necessary libraries
from pandas import read_json, DataFrame #read json file into DataFrame
from tqdm.auto import tqdm #show progress bar for loops
from numpy import zeros,float32,NaN #used in creating dummy datas and conversion
from sentence_transformers import SentenceTransformer,util #download model and also evaluate
import faiss #for index search

def convert_to_nan(x): #Data cleaning
  if x == "":
    return NaN
  return x

#import the json sent by from iqsa server
context_json_path = "/content/AI_data.json"
data = read_json(context_json_path)
data.reset_index(inplace=True)
data.columns  = ['Question', 'Answers']
data.Answers = data.Answers.apply(convert_to_nan) #cleaning the answer column
print(data.size)
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)
print(data.size)

# initialize sentence transformer model (it would download for the first time)
model = SentenceTransformer('all-MiniLM-L6-v2')

# create sentence embeddings
sentence_embeddings = model.encode(data['Question'])

#indexing
d = sentence_embeddings.shape[1] #save the len of vector
index = faiss.IndexFlatL2(d) #indexing
index.add(sentence_embeddings)

#import questionnare:list
query = list(data['Question'][[2,4,12,30,10]]) #holds all questions

#create query embeddings
k = 3
xq = model.encode(query)

#index searching (where the semantic searching occurs)
D, I = index.search(xq, k)  # search

confidences = [] #hold all confidences
similar_questions = [] #hold all similar questions
answers = [] #hold all answers

for i in tqdm(range(len(I))):
    vecs = zeros((k, d))
    for j, val in enumerate(I[i].tolist()):
        vecs[j, :] = index.reconstruct(val)

    cos_sim = util.cos_sim(xq[i].astype(float32), vecs.astype(float32))

    confidence = cos_sim[0].numpy()*100 #hold sub confidences
    confidences.append(confidence)

    similar_question = [] #hold sub similar questions
    answer_list = [] #hold sub answers
    
    for i,similar_question_index in enumerate(I[i]):
        similar_question.append(data['Question'].iloc[similar_question_index])
        answer = data[data["Question"] == data['Question'].iloc[similar_question_index]]["Answers"][similar_question_index]
        answer_list.append(answer)
        
    similar_questions.append(similar_question)
    answers.append(answer_list)

#Response DataFrame that holds the Question, Answer, Confidence,Similar_Question
response = DataFrame()
response['Question'],response['Answer'],response['Confidence'],response['Similar_Question'] = query, answers, confidences, similar_questions
response_json = response.to_json()

94
72


  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
response.head()

,Question,Answer,Confidence,Similar_Question
0,Are firewall and router configuration standard...,"[Yes, DONE, No]","[99.99999, 94.39937, 61.026978]",[Are firewall and router configuration standar...
1,Are mobile devices allowed to connect to your ...,"[Yes, Yes, Yes]","[100.000015, 55.195786, 51.867783]",[Are mobile devices allowed to connect to your...
2,Do you have a documented organogram that shows...,"[No, Yes, Yes]","[100.00002, 51.496017, 45.76556]",[Do you have a documented organogram that show...
3,Is there a formal process for approving and te...,"[No, No, Yes]","[100.000015, 97.118866, 96.97]",[Is there a formal process for approving and t...
4,Did you process the personal data of: (Articl...,"[Yes, Yes, Yes]","[100.000015, 96.96786, 65.0881]",[Did you process the personal data of: (Artic...


In [ ]:
response.Question.loc[0]

'Are firewall and router configuration standards established and implemented?;Please pick the appropriate answer'

In [ ]:
response.Answer.loc[0]

['Yes', 'DONE', 'No']

In [ ]:
response.Confidence.loc[0]

array([99.99999 , 94.39937 , 61.026978], dtype=float32)

In [ ]:
response.Similar_Question.loc[0]

['Are firewall and router configuration standards established and implemented?;Please pick the appropriate answer',
 "Are firewall and router configuration standards established and implemented?;If YES, Type 'DONE' and continue to the next question",
 'Is there a formal process for approving and testing all network connections and changes to the firewall and router configurations?;If NO, Explain your reasons detailing the following: Constraints, Compensating control, Objective, Identified Risk, Validating Controls, and Maintenance']

In [ ]:
response_json

'{"Question":{"0":"Are firewall and router configuration standards established and implemented?;Please pick the appropriate answer","1":"Are mobile devices allowed to connect to your network?","2":"Do you have a documented organogram that shows assigned roles and the internal structure of your organization or company?","3":"Is there a formal process for approving and testing all network connections and changes to the firewall and router configurations?;If NO, Explain your reasons detailing the following: Constraints, Compensating control, Objective, Identified Risk, Validating Controls, and Maintenance","4":"Did you process the personal data of:  (Article 4.1(6) and (7) );Did you process the personal data of more than 2,000 Data Subjects in the last 12 months?"},"Answer":{"0":["Yes","DONE","No"],"1":["Yes","Yes","Yes"],"2":["No","Yes","Yes"],"3":["No","No","Yes"],"4":["Yes","Yes","Yes"]},"Confidence":{"0":[99.9999923706,94.3993682861,61.0269775391],"1":[100.0000152588,55.1957855225,51.